## TESTING

In [20]:
# import libraries
import redditcleaner
import pandas as pd
import re
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis


In [21]:
# read the data
df =  pd.read_csv("redpill_sample_500.csv",
                  usecols=["body"],
                  na_filter="NaN",
                  na_values="NaN").dropna()

df["body"].dropna(inplace=True)
#df['post_preprocessed'] = df.apply(preprocess_text).str.lower()
#
#print('lemming...')
#nltk.download('wordnet')
#lemmatizer = WordNetLemmatizer()
#df['post_final'] = df['post_preprocessed'].apply(lambda post: lemmatize_words(post, lemmatizer))
#
#print('remove stopwors...')
#
#nltk.download('stopwords')
#swords = set(stopwords.words('english'))
#
#df['post_final'] = df['post_preprocessed'].apply(lambda post: remove_stopwords(post, swords))
print(df.head())

                                                body
0  We've known about the specter of censorship as...
1  Hello I just started reading about TRP theorie...
2  **CorporateLand: Don’t Kill the Job v. Quiet Q...
3  **Schrödinger’s Girlfriend**\n\n*“Girls are ne...
4  Hola amigos!! This is my current gameplan.  Pl...


In [22]:
df["body"]

0      We've known about the specter of censorship as...
1      Hello I just started reading about TRP theorie...
2      **CorporateLand: Don’t Kill the Job v. Quiet Q...
3      **Schrödinger’s Girlfriend**\n\n*“Girls are ne...
4      Hola amigos!! This is my current gameplan.  Pl...
                             ...                        
351    Recently my mate and I started a small daily h...
352    Listen when you’ve been on the road as long as...
353    Day 7 of no fap, 6 months off weed, day 4 of h...
354    This is from our elder statesman and a founder...
355    **The Importance of Manliness.**\n\nIt’s impor...
Name: body, Length: 348, dtype: object

In [23]:
text_cleaned = redditcleaner.clean(df["body"][1])

In [24]:
text_cleaned

'Hello I just started reading about TRP theories. I am still at the early chapters of the Handbook and cannot quite pin where this situation stands: I met a girl at a festival, we exchanged instagrams and now I am seeing that 80% of her stories are heavy drinking and partying. I tease her about that for a while and yesterday I asked her "I can\'t remember, but are you studying something at the moment?" My intention was to spin this around college parties and invite myself to one. She answered with "No, I don\'t study anything", but some time has passed and I was out of the mood for texting and into the mood for going to bed, so I just decided I will answer in the morning. This morning I just left her on "seen" and texted nothing back. I realized that I don\'t feel like continuing the topic and really can\'t bother with coming up with something witty to say. I think I will just text her in a few days about some other topic. How would you classify the current situation? I think it is qua

In [25]:
df["body"]

0      We've known about the specter of censorship as...
1      Hello I just started reading about TRP theorie...
2      **CorporateLand: Don’t Kill the Job v. Quiet Q...
3      **Schrödinger’s Girlfriend**\n\n*“Girls are ne...
4      Hola amigos!! This is my current gameplan.  Pl...
                             ...                        
351    Recently my mate and I started a small daily h...
352    Listen when you’ve been on the road as long as...
353    Day 7 of no fap, 6 months off weed, day 4 of h...
354    This is from our elder statesman and a founder...
355    **The Importance of Manliness.**\n\nIt’s impor...
Name: body, Length: 348, dtype: object

In [26]:
cleaned_text = []
for row in range(len(df["body"])):
    try:
        cleaned_text.append(redditcleaner.clean(df['body'][row]))
    except TypeError:
        continue
    except KeyError:
        continue

In [27]:
len(cleaned_text)

340

In [28]:
clean_df = pd.DataFrame(cleaned_text, columns=["body"])

In [29]:
def remove_urls(text):
    " removes urls"
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html(text):
    " removes html tags"
    html_pattern = re.compile('')
    return html_pattern.sub(r'', text)

def remove_emails(text):
    email_pattern = re.compile('\S*@\S*\s?')
    return email_pattern.sub(r'', text)

def remove_new_line(text):
    return re.sub('\s+', ' ', text)

def remove_non_alpha(text):
    return re.sub("[^A-Za-z]+", ' ', str(text))

def preprocess_text(text):
    #t = remove_urls(text)
    t = remove_html(text)
    #t = remove_emails(t)
    t = remove_new_line(t)
    t = remove_non_alpha(t)
    return t

def lemmatize_words(text, lemmatizer):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stopwords(text, stopwords):
    return " ".join([word for word in str(text).split() if word not in stopwords])

<>:12: DeprecationWarning: invalid escape sequence \S
<>:16: DeprecationWarning: invalid escape sequence \s
<>:12: DeprecationWarning: invalid escape sequence \S
<>:16: DeprecationWarning: invalid escape sequence \s
<>:12: DeprecationWarning: invalid escape sequence \S
<>:16: DeprecationWarning: invalid escape sequence \s
/var/folders/59/t9r6dbl91y18h_j87198fnl40000gn/T/ipykernel_10241/3997932686.py:12: DeprecationWarning: invalid escape sequence \S
  email_pattern = re.compile('\S*@\S*\s?')
/var/folders/59/t9r6dbl91y18h_j87198fnl40000gn/T/ipykernel_10241/3997932686.py:16: DeprecationWarning: invalid escape sequence \s
  return re.sub('\s+', ' ', text)


In [459]:
# read the data

clean_df['post_preprocessed'] = clean_df['body'].apply(preprocess_text).str.lower()

print('lemming...')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
clean_df['post_final'] = clean_df['post_preprocessed'].apply(lambda post: lemmatize_words(post, lemmatizer))

print('remove stopwors...')

nltk.download('stopwords')
swords = stopwords.words('english')
swords.extend(['from', 'woman',
               'women','men',
               'man','like',
               'even','go','one',
               'topic','morning','something',
               'girl','girls', 'guy', 'give',
               'would','things',
               'know','really',
               'never','hello',
               'first','every','lot','much','make',
               'want','life','good','x',
               'people','going','getting','saying',
               'day', 'get',
               'say','way',
               'guys', 'think',
               'feel','also',
               'see', 'need', 'thing',
               'b', 'many',
               'take','let',
               'still', 'fuck',
               'could','may',
               'said', 'back', 'better', 'always', 'right',
               'time', 'shit','around', 'fucking', 'work',
               'sexual', 'got', 'years', 'post', 'point',
               'self', 'world',
               'ever','trying', 'talking',
               'long', 'year', 'tell',
               'another', 'someone', 'look',
               'everything', 'anything', 'nothing',
               'find', 'enough', 'actually',
               'keep', 'well', 'best', 'number',
               'sex', 'game', 'mind',
               'talk', 'new', 'next', 'love'])
swords =set(swords)
clean_df['post_final'] = clean_df['post_preprocessed'].apply(lambda post: remove_stopwords(post, swords))
print(clean_df.head())

lemming...


[nltk_data] Downloading package wordnet to /Users/tugberk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


remove stopwors...
                                                body  \
0  We've known about the specter of censorship as...   
1  Hello I just started reading about TRP theorie...   
2  CorporateLand: Don’t Kill the Job v. Quiet Qui...   
3  Schrödinger’s Girlfriend “Girls are never sing...   
4  Hola amigos This is my current gameplan. Pleas...   

                                   post_preprocessed  \
0  we ve known about the specter of censorship as...   
1  hello i just started reading about trp theorie...   
2  corporateland don t kill the job v quiet quitt...   
3  schr dinger s girlfriend girls are never singl...   
4  hola amigos this is my current gameplan please...   

                                          post_final  
0  known specter censorship red pill censorship f...  
1  started reading trp theories early chapters ha...  
2  corporateland kill job v quiet quitting two di...  
3  schr dinger girlfriend single relationships sa...  
4  hola amigos current gameplan 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tugberk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [460]:
clean_df.sample(4)

,body,post_preprocessed,post_final
44,I started off thinking trp was about girls and...,i started off thinking trp was about girls and...,started thinking trp pussy far beyond honestly...
113,Casey Anthony epitomized something that I coul...,casey anthony epitomized something that i coul...,casey anthony epitomized quite put finger sat ...
289,"Most modern men (men is a strong word, maybe f...",most modern men men is a strong word maybe fem...,modern strong word maybe femboy accurate addic...
198,"Hello brothers, I’m not sure any of you may re...",hello brothers i m not sure any of you may rem...,brothers sure remember story almost two since ...


In [461]:
posts = [x.split(' ') for x in clean_df['post_final']]
#p2 = []
#for i in range(len(posts)):
 #   p2.append(posts[posts[i] not in ['women','men','man','like','even','go','one', 'topic','morning','something']])
id2word = corpora.Dictionary(posts)
corpus_tf = [id2word.doc2bow(text) for text in posts]
#print(corpus_tf[0])

In [462]:
tfidf = models.TfidfModel(corpus_tf)
corpus_tfidf = tfidf[corpus_tf]
#print(corpus_tfidf[0])

In [463]:
model = LdaMulticore(corpus=corpus_tf,id2word = id2word, num_topics = 5,
                    chunksize=50,
                     random_state = 0)

coherence = CoherenceModel(model = model, texts = posts, dictionary = id2word, coherence = 'u_mass')

print(coherence.get_coherence())
#print(model.show_topics())

-1.1136559547404297


In [466]:
for i in range(5):
    print("topic {} \n".format(i),model.show_topic(topicid=i,topn=10))

topic 0 
 [('approach', 0.00256927), ('high', 0.0019090173), ('night', 0.0018843431), ('frame', 0.0018474186), ('social', 0.0018260332), ('different', 0.0018228294), ('wants', 0.0017581327), ('alpha', 0.0017472613), ('might', 0.0017390577), ('little', 0.0017377133)]
topic 1 
 [('try', 0.0020712262), ('without', 0.0019180439), ('social', 0.0018719806), ('us', 0.0018109387), ('relationship', 0.0017857852), ('partners', 0.001703858), ('else', 0.0016711933), ('alpha', 0.001654076), ('red', 0.0016463625), ('since', 0.0016174719)]
topic 2 
 [('us', 0.0024020176), ('try', 0.0023717063), ('red', 0.002290104), ('approach', 0.002251057), ('start', 0.0021032104), ('pill', 0.0018654057), ('use', 0.0018206816), ('come', 0.0018175311), ('relationship', 0.0018103085), ('friends', 0.001782004)]
topic 3 
 [('made', 0.00214781), ('attractive', 0.0021055061), ('red', 0.0019165752), ('money', 0.0019107821), ('pill', 0.0019084745), ('wanted', 0.0018895182), ('approach', 0.0018783741), ('making', 0.00184726

In [467]:
data_dict = {'dominant_topic':[], 'perc_contribution':[], 'topic_keywords':[]}

for i, row in enumerate(model[corpus_tf]):
    #print(i)
    row = sorted(row, key=lambda x: x[1], reverse=True)
    #print(row)
    for j, (topic_num, prop_topic) in enumerate(row):
        wp = model.show_topic(topic_num)
        topic_keywords = ", ".join([word for word, prop in wp])
        data_dict['dominant_topic'].append(int(topic_num))
        data_dict['perc_contribution'].append(round(prop_topic, 3))
        data_dict['topic_keywords'].append(topic_keywords)
        #print(topic_keywords)
        break

df_topics = pd.DataFrame(data_dict)

contents = pd.Series(posts)

df_topics['post'] = clean_df['body']
df_topics.sample(20)

,dominant_topic,perc_contribution,topic_keywords,post
130,2,0.996,"us, try, red, approach, start, pill, use, come...",Let me start from the begining. this post is a...
261,0,0.995,"approach, high, night, frame, social, differen...",It's been 3 years since my last TRP post. Figu...
230,4,0.996,"alpha, relationship, end, pill, friend, makes,...","So, I'll keep this brief. There's probably a b..."
209,4,0.793,"alpha, relationship, end, pill, friend, makes,...",Good afternoon TRP. Today I want to cover some...
333,0,0.970,"approach, high, night, frame, social, differen...",There's a saying in sales and Internet Marketi...
150,2,0.999,"us, try, red, approach, start, pill, use, come...","Intro: You’ve been working out for a while, w..."
219,2,0.512,"us, try, red, approach, start, pill, use, come...",&x200B; Attraction. This single concept gets m...
153,2,0.497,"us, try, red, approach, start, pill, use, come...","Bear with me through the intro, it's needed fo..."
306,2,0.975,"us, try, red, approach, start, pill, use, come...",I had posted earlier in the asktrp subreddit a...
290,4,0.997,"alpha, relationship, end, pill, friend, makes,...",My calculus teacher was ahead of his time: > T...


In [474]:
for i in range(5):
    print("topic {} \n".format(i),df_topics[df_topics["dominant_topic"] == i]["post"].sample(5), "\n")

topic 0 
 284    A lot has been written on the topic of No Nut ...
251    One of my favourite TRP contributors is u/Wouj...
31     In today's society the most important thing to...
98     1. You are gonna be timid and fearful in your ...
122    This is a breakdown of a post from the Stoicis...
Name: post, dtype: object 

topic 1 
 110    For an introduction, have been a follower, rea...
11     Through EEO/AA, men have been bleeding their t...
259    Gonna be a lenhy post so sit down, relax and g...
46     TL;DR: Left my girl, we grew apart. My "RP'd f...
277    Diwali (the festival of lights, if you're inte...
Name: post, dtype: object 

topic 2 
 231    When listening to Red Pill content, a lot of t...
301    Now that I've got your attention, humor me and...
184    Put Yourself First. Why Being an Asshole is a ...
268    There has been far too many noobers around the...
311    There are a lot of men who find the Red Pill b...
Name: post, dtype: object 

topic 3 
 217    A guy in this 